# <hr style="clear: both" />

# Running Spark in YARN-client mode

This notebook demonstrates how to set up a SparkContext that uses SURFsara's Hadoop cluster: [YARN resourcemanager](http://head05.hathi.surfsara.nl:8088/cluster) (note you will need to be authenticated via kerberos on your machine to visit the resourcemanager link) for executors.

First initialize kerberos via a Jupyter terminal. 
In the terminal execute: <BR>
<i>kinit -k -t data/robertop.keytab robertop@CUA.SURFSARA.NL</i><BR>
Print your credentials:


In [ ]:
! klist

In [2]:
print sc

# <hr style="clear: both" />

# Now you can run your code

Pick a clustering algorithm (name of the file that provides a classify(x,y [,threshold]) function)

In [3]:
execfile('../spark-scripts/conventions.py')
execfile('../spark-scripts/eval.py')
execfile('../spark-scripts/implicitPlaylistAlgoFunctions.py')
execfile('../spark-scripts/implicitPlaylistAlgoMain.py')

CLUSTER_ALGO = 'jaccardBase'

execfile('../spark-scripts/' + CLUSTER_ALGO + '.py')


# Reading the conf file

In [4]:
import json
import copy

BASE_PATH = "/mnt/space/mattia"

conf = {}

conf['split'] = {}
conf['split']['reclistSize'] = 100
conf['split']['callParams'] = {}
conf['split']['excludeAlreadyListenedTest'] = True
conf['split']['name'] = 'SenzaRipetizioni_1'
conf['split']['split'] = conf['split']['name']
conf['split']['minEventsPerUser'] = 5
conf['split']['inputData'] = BASE_PATH + '/' + CLUSTER_ALGO + '.split/SenzaRipetizioni_1'
#conf['split']['inputData'] = 's3n://contentwise-research-poli/30musicdataset/newFormat/relations/sessions.idomaar'
conf['split']['bucketName'] = BASE_PATH
conf['split']['percUsTr'] = 0.05
conf['split']['ts'] = int(0.75 * (1421745857 - 1390209860) + 1390209860) - 10000
conf['split']['minEventPerSession'] = 5
conf['split']['onlineTrainingLength'] = 5
conf['split']['GTlength'] = 1
conf['split']['minEventPerSessionTraining'] = 10
conf['split']['minEventPerSessionTest'] = 11
conf['split']['mode'] = 'session'
conf['split']['forceSplitCreation'] = False
conf['split']["prop"] = {'reclistSize': conf['split']['reclistSize']}
conf['split']['type'] = None
conf['split']['out'] = BASE_PATH + '/' + CLUSTER_ALGO + '.split'
conf['split']['location'] = '30Mdataset/relations/sessions'

conf['evaluation'] = {}
conf['evaluation']['metric'] = {}
conf['evaluation']['metric']['type'] = 'recall'
conf['evaluation']['metric']['prop'] = {}
conf['evaluation']['metric']['prop']['N'] = [1,2,5,10,15,20,25,50,100]
conf['evaluation']['name'] = 'recall@N'

conf['general'] = {}
conf['general']['clientname'] = CLUSTER_ALGO + '.split'
conf['general']['bucketName'] = BASE_PATH
conf['general']['tracksPath'] = '30Mdataset/entities/tracks.idomaar.gz'

conf['algo'] = {}
conf['algo']['name'] = CLUSTER_ALGO
conf['algo']['props'] = {}
# ***** EXAMPLE OF CONFIGURATION *****#
conf['algo']['props']["sessionJaccardShrinkage"] = 5
conf['algo']['props']["clusterSimilarityThreshold"] = 0.1
conf['algo']['props']["expDecayFactor"] = 0.7
# ****** END EXAMPLE ****************#

# <hr style="clear: both" />
# Pick the list of songs ad create clusters


In [4]:
import json
import string

def my_replace_punct(x):
    ret = ""
    for i in x:
        if i == '+':
            ret += ' '
        else:
            ret += i
    return ret

tracksRDD = sc.textFile(BASE_PATH + '/30Mdataset/entities/tracks.idomaar.gz')
tracksRDD = tracksRDD.map(lambda x: x.split('\t')).map(lambda x: (x[1], json.loads(x[3])['name'].split('/') ) )
tracksRDD = tracksRDD.map(lambda x: (x[0], " ".join( (x[1][0], x[1][2]) ) ))
tracksRDD = tracksRDD.map(lambda x : (x[0], my_replace_punct(x[1])))
tracksRDD = tracksRDD.map(lambda x: (x[0], tokenize_song(x[1]), x[1]))


sampleRDD = tracksRDD.take(5000)
sampleRDD = sc.parallelize(sampleRDD)

tracksIdsRDD = tracksRDD.map(lambda x: (x[0], [x[0]]))

tracksIdsRDD.take(3)


[(u'0', [u'0']), (u'1', [u'1']), (u'2', [u'2'])]

# <hr style="clear: both" />

Reduce the quantity of data by building RDD {word -> songs}.
For each word keep only couples of songs that match.


In [5]:
#Build an RDD with ('word' -> (id, name))
wordsRDD = sampleRDD.flatMap(lambda x: [(i, (x[0], x[2])) for i in x[1]] )
wordsRDD.take(3)

#Group by 'word' and keep only the ones with more then 1 song
wordsRDD = wordsRDD.groupByKey().mapValues(list).filter(lambda x: len(x[1]) > 1)

#Compute a cartesian product for each list of songs with a common word
def filtered_cartesian(x):
    equal_couples = set()
    for i in range(len(x[1])):
        a = x[1][i]
        id_a = x[1][i][0]
        name_a = x[1][i][1]
        
        for j in range(i):
            b = x[1][j]
            id_b = x[1][j][0]
            name_b = x[1][j][1]
            if id_a != id_b:
                if classify(name_a, name_b):
                    equal_couples.add((a,b))
                    
    return (x[0], tuple(equal_couples))

coupleRDD = wordsRDD.map(filtered_cartesian).filter(lambda x: len(x[1]) > 1)
coupleRDD.take(3)

[(u'daydreamer',
  (((u'2852', u'10 Years 11.00 AM (Daydreamer)'),
    (u'2851', u'10 Years 11:00 AM (Daydreamer)')),
   ((u'2851', u'10 Years 11:00 AM (Daydreamer)'),
    (u'2850', u'10 Years 11-00 AM (Daydreamer)')),
   ((u'2852', u'10 Years 11.00 AM (Daydreamer)'),
    (u'2850', u'10 Years 11-00 AM (Daydreamer)')))),
 (u'and',
  (((u'3081', u'112 Peaches And Cream'),
    (u'2997', u'112 112 - Peaches and Cream')),
   ((u'2859', u'10 Years ... And All The Other Colors'),
    (u'2858', u'10 Years ...And All the Other Colors')))),
 (u'stones',
  (((u'4154', u'12 Stones Lie to Me (Acoustic)'),
    (u'4153', u'12 Stones Lie to me - acoustic')),
   ((u'4126', u'12 Stones Bulletproof_'),
    (u'4125', u'12 Stones Bulletproof'))))]

# <hr style="clear: both" />
Flip the dataset and map each song to the couples it belongs to. 
Group by key and for each song you have a cluster!


In [6]:
#Flatmap the list of couples
flattedCoupleRDD = coupleRDD.flatMap(lambda x: [i for i in x[1]])
#For each couple, for each song, yield song->couple
flattedCoupleRDD = flattedCoupleRDD.flatMap(lambda x: ((i[0], (x[0], x[1]) )for i in x) )


#Group by key (song). Each song has now one cluster
def merge_couples(x, y):
    return list(set(x) | set(y))

songClusterRDD = flattedCoupleRDD.reduceByKey(merge_couples).map(lambda x: (x[0], [i[0] for i in x[1]]))
songClusterRDD.take(30)

[(u'2859', [u'2859', u'2858']),
 (u'2648', [u'2639', u'2648']),
 (u'3603', [u'3602', u'3603']),
 (u'3610', [u'3611', u'3610']),
 (u'2858', [u'2859', u'2858']),
 (u'3602', [u'3602', u'3603']),
 (u'3611', [u'3611', u'3610']),
 (u'3612', [u'3613', u'3612']),
 (u'4217', [u'4217', u'4246']),
 (u'3613', [u'3613', u'3612']),
 (u'4216', [u'4245', u'4216']),
 (u'3614', [u'3615', u'3614']),
 (u'2639', [u'2639', u'2648']),
 (u'2961', [u'2959', u'2961']),
 (u'2638', [u'2638', u'2680']),
 (u'3615', [u'3615', u'3614']),
 (u'3775', [u'3775', u'3774']),
 (u'3774', [u'3775', u'3774']),
 (u'2851', [u'2851', u'2850', u'2852']),
 (u'2640', [u'2670', u'2640']),
 (u'4246', [u'4217', u'4246']),
 (u'2868', [u'2868', u'2869']),
 (u'2850', [u'2851', u'2850', u'2852']),
 (u'2869', [u'2868', u'2869']),
 (u'3623', [u'3623', u'3622']),
 (u'2448', [u'2448', u'2449']),
 (u'2852', [u'2851', u'2850', u'2852']),
 (u'4245', [u'4245', u'4216']),
 (u'3622', [u'3623', u'3622']),
 (u'2449', [u'2448', u'2449'])]

# <hr style="clear: both" />
Complete the clusters with all the other songs. 
Then we need to map cluster to songs to have new IDs.

In [7]:
#In this way we obtain a complete RDD with song -> group of songs
def reduce_to_biggest(x, y):
    bigger = x if len(x) > len(y) else y
    result = sorted(bigger)
    return result
           
unionRDD = songClusterRDD.union(tracksIdsRDD).reduceByKey(reduce_to_biggest)
unionRDD.take(100)


[(u'3187564', [u'3187564']),
 (u'839798', [u'839798']),
 (u'3265519', [u'3265519']),
 (u'1570601', [u'1570601']),
 (u'172037', [u'172037']),
 (u'228052', [u'228052']),
 (u'2672583', [u'2672583']),
 (u'3597028', [u'3597028']),
 (u'2685368', [u'2685368']),
 (u'2653961', [u'2653961']),
 (u'4972098', [u'4972098']),
 (u'931855', [u'931855']),
 (u'684237', [u'684237']),
 (u'2294826', [u'2294826']),
 (u'1770605', [u'1770605']),
 (u'1971070', [u'1971070']),
 (u'2103417', [u'2103417']),
 (u'1837140', [u'1837140']),
 (u'2413655', [u'2413655']),
 (u'3601843', [u'3601843']),
 (u'1902200', [u'1902200']),
 (u'4541278', [u'4541278']),
 (u'3206162', [u'3206162']),
 (u'776530', [u'776530']),
 (u'1179403', [u'1179403']),
 (u'1751011', [u'1751011']),
 (u'127470', [u'127470']),
 (u'4558204', [u'4558204']),
 (u'3792040', [u'3792040']),
 (u'2901109', [u'2901109']),
 (u'2462396', [u'2462396']),
 (u'4470544', [u'4470544']),
 (u'1497322', [u'1497322']),
 (u'4983478', [u'4983478']),
 (u'4424928', [u'4424928']),

# <hr style="clear: both" />
We have song -> cluster. We map inversly (cluster -> song) and group by key (cluster).
Finally we zip with index and obtain new IDs.

In [23]:
#Flip the mapping as cluster->song
clusterSongsRDD = unionRDD.map(lambda x: (' '.join(x[1]), x[0])).groupByKey().mapValues(list)
clusterSongsRDD = clusterSongsRDD.zipWithIndex().map(lambda x: (x[1], x[0][1]))
clusterSongsRDD.take(30)

[(0, [u'3187564']),
 (1, [u'839798']),
 (2, [u'3265519']),
 (3, [u'1570601']),
 (4, [u'228052']),
 (5, [u'2672583']),
 (6, [u'34547']),
 (7, [u'2653961']),
 (8, [u'4972098']),
 (9, [u'931855']),
 (10, [u'684237']),
 (11, [u'2294826']),
 (12, [u'1770605']),
 (13, [u'1971070']),
 (14, [u'2103417']),
 (15, [u'1837140']),
 (16, [u'2413655']),
 (17, [u'3601843']),
 (18, [u'1902200']),
 (19, [u'1547135']),
 (20, [u'4541278']),
 (21, [u'3206162']),
 (22, [u'776530']),
 (23, [u'486721']),
 (24, [u'1879897']),
 (25, [u'1751011']),
 (26, [u'127470']),
 (27, [u'2901109']),
 (28, [u'2462396']),
 (29, [u'4470544'])]

In [24]:
clusterSongsRDD.take(10)

[(0, [u'3187564']),
 (1, [u'839798']),
 (2, [u'3265519']),
 (3, [u'1570601']),
 (4, [u'228052']),
 (5, [u'2672583']),
 (6, [u'34547']),
 (7, [u'2653961']),
 (8, [u'4972098']),
 (9, [u'931855'])]

# <hr style="clear: both" />
Save the RDD (newID, songs) in "/mnt/space/mattia/clusters/[ALGORITHM]"

In [26]:
#Save clustering
clusterSongsRDD.saveAsPickleFile(BASE_PATH + '/clusters/' + CLUSTER_ALGO)


Split the data

In [ ]:
execfile('../spark-scripts/splitCluster2.py')
splitter(conf)

# Substitute trackID with clusterID

Create RDD with song -> clusterID

In [5]:
clusterSongsFileRDD = sc.pickleFile(BASE_PATH + '/clusters/' + CLUSTER_ALGO)
clusterSongsFileRDD.map(lambda x: x.split('\n'))
songToClusterRDD = clusterSongsFileRDD.flatMap(lambda x: [(int(i), x[0]) for i in x[1]] )

songToClusterRDD.take(3)

[(4479945, 2364462), (4436240, 2364463), (3181144, 2364464)]

It's time to substitute songs with their cluster

In [7]:
import json
execfile('../spark-scripts/utilsCluster2.py')
train, test = loadDataset(conf)

train.take(1)

[u'{"linkedinfo": {"subjects": [{"type": "user", "id": 14598}], "objects": [{"playratio": 1.01, "playstart": 0, "action": "play", "playtime": 194, "type": "track", "id": 276164}, {"playratio": 1.0, "playstart": 194, "action": "play", "playtime": 179, "type": "track", "id": 276174}, {"playratio": 1.0, "playstart": 373, "action": "play", "playtime": 205, "type": "track", "id": 276202}, {"playratio": 0.77, "playstart": 578, "action": "play", "playtime": 288, "type": "track", "id": 276224}, {"playratio": 1.02, "playstart": 866, "action": "play", "playtime": 158, "type": "track", "id": 276240}, {"playratio": 1.01, "playstart": 1024, "action": "play", "playtime": 183, "type": "track", "id": 276192}, {"playratio": 1.0, "playstart": 1207, "action": "play", "playtime": 193, "type": "track", "id": 276163}, {"playratio": 1.0, "playstart": 1400, "action": "play", "playtime": 217, "type": "track", "id": 276255}, {"playratio": 0.85, "playstart": 1617, "action": "play", "playtime": 282, "type": "trac

In [8]:
def flat_map_tracks_ids(x):
    objects = x['linkedinfo']['objects']
    result = []
    for i in range(len(objects)):
        result.append( (objects[i]['id'], (i, x)) )
    return result

trainFlat = train.map(lambda x: json.loads(x)).flatMap(flat_map_tracks_ids)
trainJoin = trainFlat.join(songToClusterRDD)
trainJoin.take(1)

[(3849219,
  ((4,
    {u'id': u'616821',
     u'linkedinfo': {u'objects': [{u'action': u'play',
        u'id': 3849194,
        u'playratio': 1.0,
        u'playstart': 0,
        u'playtime': 259,
        u'type': u'track'},
       {u'action': u'play',
        u'id': 3849191,
        u'playratio': 0.36,
        u'playstart': 259,
        u'playtime': 102,
        u'type': u'track'},
       {u'action': u'play',
        u'id': 3849191,
        u'playratio': 1.0,
        u'playstart': 361,
        u'playtime': 286,
        u'type': u'track'},
       {u'action': u'play',
        u'id': 3849214,
        u'playratio': 1.0,
        u'playstart': 647,
        u'playtime': 448,
        u'type': u'track'},
       {u'action': u'play',
        u'id': 3849219,
        u'playratio': 1.0,
        u'playstart': 1095,
        u'playtime': 334,
        u'type': u'track'},
       {u'action': u'play',
        u'id': 3849193,
        u'playratio': 1.01,
        u'playstart': 1429,
        u'playtime': 113

In [12]:
trainSub = trainJoin.map(lambda x: (json.dumps(x[1][0][1]), (x[1][0][0], x[1][1])))
trainAgg = trainSub.groupByKey().mapValues(list)
trainAgg.take(1)

[('{"linkedinfo": {"subjects": [{"type": "user", "id": 24296}], "objects": [{"playratio": 1.18, "playstart": 0, "action": "play", "playtime": 246, "type": "track", "id": 190842}, {"playratio": 1.18, "playstart": 0, "action": "play", "playtime": 246, "type": "track", "id": 190842}, {"playratio": 1.18, "playstart": 0, "action": "play", "playtime": 246, "type": "track", "id": 190842}, {"playratio": 1.18, "playstart": 0, "action": "play", "playtime": 246, "type": "track", "id": 190842}, {"playratio": 1.18, "playstart": 0, "action": "play", "playtime": 246, "type": "track", "id": 190842}, {"playratio": 1.18, "playstart": 0, "action": "play", "playtime": 246, "type": "track", "id": 190842}, {"playratio": 1.39, "playstart": 246, "action": "play", "playtime": 344, "type": "track", "id": 190816}, {"playratio": 1.39, "playstart": 246, "action": "play", "playtime": 344, "type": "track", "id": 190816}, {"playratio": 1.39, "playstart": 246, "action": "play", "playtime": 344, "type": "track", "id": 

In [15]:
def plug_clusters(x):
    row_dic = json.loads(x[0])
    to_be_plugged = x[1]
    for i in to_be_plugged:
        index = i[0]
        cl_id = i[1]
        row_dic['linkedinfo']['objects'][index]['id'] = cl_id
    return json.dumps(row_dic)
    
trainRDD = trainAgg.map(plug_clusters)
trainRDD.take(100)

['{"linkedinfo": {"subjects": [{"type": "user", "id": 24296}], "objects": [{"playratio": 1.18, "playstart": 0, "action": "play", "playtime": 246, "type": "track", "id": 4432528}, {"playratio": 1.18, "playstart": 0, "action": "play", "playtime": 246, "type": "track", "id": 4432528}, {"playratio": 1.18, "playstart": 0, "action": "play", "playtime": 246, "type": "track", "id": 4432528}, {"playratio": 1.18, "playstart": 0, "action": "play", "playtime": 246, "type": "track", "id": 4432528}, {"playratio": 1.18, "playstart": 0, "action": "play", "playtime": 246, "type": "track", "id": 4432528}, {"playratio": 1.18, "playstart": 0, "action": "play", "playtime": 246, "type": "track", "id": 4432528}, {"playratio": 1.39, "playstart": 246, "action": "play", "playtime": 344, "type": "track", "id": 3850238}, {"playratio": 1.39, "playstart": 246, "action": "play", "playtime": 344, "type": "track", "id": 3850238}, {"playratio": 1.39, "playstart": 246, "action": "play", "playtime": 344, "type": "track",

In [ ]:
testFlat = test.map(lambda x: json.loads(x)).flatMap(flat_map_tracks_ids)
testJoin = testFlat.join(songToClusterRDD)

testSub = testJoin.map(lambda x: (json.dumps(x[1][0][1]), (x[1][0][0], x[1][1])))
testAgg = testSub.groupByKey().mapValues(list)

testRDD = testAgg.map(plug_clusters)
testRDD.take(100)

In [ ]:
from os import path
basePath = path.join(conf['general']['bucketName'], conf['general']['clientname'])
splitPath = path.join(basePath, conf['split']['name'])

clusterSimList = [0.1]
sessionJaccardShrinkageList = [5]
expDecayList = [0.7]

for exclude in [True]:
    conf['split']['excludeAlreadyListenedTest'] = str(exclude)
    #conf['split']['name'] = 'giroCompletoTestMultipleConfs_exclude%s' % exclude
    #splitter(conf)
    #train, test = loadDataset(conf)
    #train.cache()
    #test.cache()
    
    for sessionJaccardShrinkage in sessionJaccardShrinkageList:
        conf['algo']['props']["sessionJaccardShrinkage"] = sessionJaccardShrinkage
        
        for clusterSim in clusterSimList:
            conf['algo']['props']["clusterSimilarityThreshold"] = clusterSim
            
            playlists = extractImplicitPlaylists(train, conf).cache()
            
            for expDecay in expDecayList:
                conf['algo']['props']["expDecayFactor"] = expDecay
                conf['algo']['name'] = CLUSTER_ALGO + '_ImplicitPlaylist_shk_%d_clustSim_%.3f_decay_%.3f' % \
                    (sessionJaccardShrinkage, clusterSim, expDecay )

                recJsonRDD = executeImplicitPlaylistAlgo(playlists, test, conf)
                try:
                    saveRecommendations(conf, recJsonRDD, overwrite=True)
                    try:
                        computeMetrics(conf)
                    except:
                        print 'Error in computing metrics'
                except:
                    print 'Error in saving recommndations'
                    try:
                        computeMetrics(conf)
                    except:
                        print 'Error in computing metrics'